In [1]:
import pandas as pd
import numpy as np
import os

import torch
import torchvision



import transformers
import simpletransformers

from simpletransformers.language_modeling import LanguageModelingModel

from transformers import BertForSequenceClassification, BertTokenizer, BertForMaskedLM

from sklearn.metrics.pairwise import cosine_similarity, paired_euclidean_distances
from sklearn.metrics.pairwise import euclidean_distances

I0518 14:38:46.559072 140465605080832 file_utils.py:41] PyTorch version 1.5.0 available.
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site

In [2]:
dataFolder = '/data1/roshansk/covid_data/'
fileList = os.listdir(dataFolder)

In [4]:
fileList

['messages_cm_mar1_apr23_noRT.csv',
 'messages_cm_mar1_apr23.csv',
 'covid19_msgs_2020_02.csv']

In [5]:
df = pd.read_csv(os.path.join(dataFolder, fileList[0]))

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (20,21,30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
len(df)

2930933

In [13]:
df['RT'] = df.message.apply(lambda x : 1 if x[:2]=='RT' else 0)

In [14]:
df.RT.value_counts()

1    73063
0    26937
Name: RT, dtype: int64

In [8]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', output_hidden_states= True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

print('Model loaded')

I0512 10:05:08.288321 139728279049984 configuration_utils.py:283] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/roshansk/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0512 10:05:08.290026 139728279049984 configuration_utils.py:319] Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2i

Model loaded


#### Embedding relationship tests

In [16]:
sentence1 = "I want to sit on that table"
token1 = tokenizer.encode(sentence1)
decode1 = tokenizer.decode(token1).split(" ")

logits, hidden_states = model(torch.Tensor(token1).unsqueeze(0).long())

sent1 = hidden_states[0].squeeze(0).detach().cpu().numpy()
sent1_ = hidden_states[-1].squeeze(0).detach().cpu().numpy()

In [41]:
sentence1 = "I want to sit on that table"
token1 = tokenizer.encode(sentence1)
decode1 = tokenizer.decode(token1).split(" ")

logits, hidden_states = model(torch.Tensor(token1).unsqueeze(0).long())

sent1 = hidden_states[0].squeeze(0).detach().cpu().numpy()
sent1_ = hidden_states[-1].squeeze(0).detach().cpu().numpy()

In [42]:
sentence2 = "The books are placed on the table"
token2 = tokenizer.encode(sentence2)
decode2 = tokenizer.decode(token2).split(" ")
logits, hidden_states = model(torch.Tensor(token2).unsqueeze(0).long())

sent2 = hidden_states[0].squeeze(0).detach().cpu().numpy()
sent2_ = hidden_states[-1].squeeze(0).detach().cpu().numpy()

In [35]:
sentence2 = "Lets table this discussion"
token2 = tokenizer.encode(sentence2)
decode2 = tokenizer.decode(token2).split(" ")
logits, hidden_states = model(torch.Tensor(token2).unsqueeze(0).long())

sent2 = hidden_states[0].squeeze(0).detach().cpu().numpy()
sent2_ = hidden_states[-1].squeeze(0).detach().cpu().numpy()

In [44]:
sentence2 = "The living room has a large table with few chairs"
token2 = tokenizer.encode(sentence2)
decode2 = tokenizer.decode(token2).split(" ")
logits, hidden_states = model(torch.Tensor(token2).unsqueeze(0).long())

sent2 = hidden_states[0].squeeze(0).detach().cpu().numpy()
sent2_ = hidden_states[-1].squeeze(0).detach().cpu().numpy()

In [46]:
sentence2 = "Lets table the discussion for now and put these books on the table"
token2 = tokenizer.encode(sentence2)
decode2 = tokenizer.decode(token2).split(" ")
logits, hidden_states = model(torch.Tensor(token2).unsqueeze(0).long())

sent2 = hidden_states[0].squeeze(0).detach().cpu().numpy()
sent2_ = hidden_states[-1].squeeze(0).detach().cpu().numpy()

In [47]:
for i in range(len(decode1)):
    word1 = decode1[i]

    
    if word1 != 'table':
        continue
        
    print(f"{word1}")
    
    for j in range(len(decode2)):
        word2 = decode2[j]
        dist = euclidean_distances(sent1_[i].reshape(1,-1), sent2_[j].reshape(1,-1))
        print(f"{word2} , {dist[0][0]}")
    print("------")
        

table
[CLS] , 19.208110809326172
lets , 19.866884231567383
table , 16.655344009399414
the , 18.049161911010742
discussion , 14.780564308166504
for , 17.403345108032227
now , 19.56454086303711
and , 17.332332611083984
put , 17.118425369262695
these , 18.02132797241211
books , 15.759896278381348
on , 17.43817138671875
the , 18.873056411743164
table , 11.894767761230469
[SEP] , 19.93589973449707
------


In [15]:
for i in range(len(decode1)):
    word1 = decode1[i]

    
    if word1 != 'table':
        continue
        
    print(f"{word1}")
    
    for j in range(len(decode2)):
        word2 = decode2[j]
        dist = euclidean_distances(sent1_[i].reshape(1,-1), sent2_[j].reshape(1,-1))
        print(f"{word2} -> {dist[0]}")
    print("------")
        

table
[CLS] -> [19.20811]
lets -> [19.866884]
table -> [16.655344]
the -> [18.049162]
discussion -> [14.780564]
for -> [17.403345]
now -> [19.56454]
and -> [17.332333]
put -> [17.118425]
these -> [18.021328]
books -> [15.759896]
on -> [17.438171]
the -> [18.873056]
table -> [11.894768]
[SEP] -> [19.9359]
------


In [53]:
sent1_0

array([[ 0.16855057, -0.28576732, -0.32612562, ..., -0.02757058,
         0.03825327,  0.16399528],
       [-0.35573578,  0.95750636, -1.3448268 , ...,  0.5971607 ,
         0.7391579 ,  0.25808817],
       [-0.98275685,  0.4850043 , -0.74877006, ...,  0.60817134,
         0.9695404 , -0.42965782],
       [-1.1714747 , -1.0952786 , -0.7364164 , ...,  0.50561   ,
        -0.6454608 ,  0.75769067],
       [-0.36429545, -0.16171779,  0.09017371, ..., -0.17849147,
         0.12817715, -0.04511606]], dtype=float32)

In [54]:
sent2_0

array([[ 0.16855057, -0.28576732, -0.32612562, ..., -0.02757058,
         0.03825327,  0.16399528],
       [-0.4164709 ,  0.23321255, -0.65267146, ...,  0.86469525,
        -0.14918818, -0.01466047],
       [-1.7195247 , -0.48560476,  0.39680693, ...,  0.4649588 ,
        -0.8625533 ,  0.70038414],
       [ 0.49153116, -0.76267207,  0.08711345, ...,  0.5580665 ,
        -0.38935202,  0.46139798],
       [-0.5772839 ,  0.773778  , -1.1550461 , ...,  0.49831286,
         0.33618513,  0.45114946],
       [-0.32507333, -0.3187855 , -0.11631773, ..., -0.39602298,
         0.41119915, -0.07755204]], dtype=float32)

In [66]:
scores = []
for i in range(6):
    dist = euclidean_distances(sent1_12[1].reshape(1,-1), sent2_12[i].reshape(1,-1))
    scores.append(dist)
    


In [67]:
dist = euclidean_distances(sent1_0[1].reshape(1,-1), sent2_0[i].reshape(1,-1))

In [69]:
scores

[array([[21.20042]], dtype=float32),
 array([[21.033146]], dtype=float32),
 array([[20.308558]], dtype=float32),
 array([[20.273989]], dtype=float32),
 array([[6.463085]], dtype=float32),
 array([[19.155994]], dtype=float32)]

## Language Modeling

In [94]:
train_args = {
    "output_dir":'CovidBertModel',
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    'fp16':False,
    'num_train_epochs':2
}

model = LanguageModelingModel('bert', 'bert-base-uncased', args=train_args)

I0511 01:09:10.835188 139812352890624 tokenization_utils.py:420] Model name 'CovidBertModel' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming 'CovidBertModel' is a path, a model identifier, or url to a directory containing tokenizer files.
I0511 01:09:10.837392 139812352890624 tokenization_utils.py:449] Didn't find file CovidBertModel/added_tokens.json. We won't load it.
I0511 01:09:10.838767 139812352890624 tokenization_utils.py:502] loading file CovidBertM

In [39]:
trainDf = df[['message','user_id']]
trainDf.columns = ['text','labels']

In [40]:
trainDf.head()

,text,labels
0,loving how i predicted the #coronavirus in my ...,790013818999074820
1,RT @ShotsRangOut: FNC gave this fool a platfor...,732620465735405569
2,The markets is hemorrhaging #trump and his pal...,320608440
3,"RT @AdaKirschner: Heads up, y'all.nnThe Nation...",2186554418
4,RT @ExJWStephen: Jehovah's Witnesses have offi...,2893026960


In [42]:
trainDf.to_csv('CovidTrainDf.csv',index=False, index_label=False)

In [43]:
model.train_model('CovidTrainDf.csv')

I0509 21:45:28.468431 140689433306880 language_modeling_utils.py:177]  Creating features from dataset file at cache_dir/


I0509 21:45:36.366803 140689433306880 language_modeling_utils.py:225]  Saving features into cached file cache_dir/bert_cached_lm_126_CovidTrainDf.csv
I0509 21:45:36.503617 140689433306880 language_modeling_model.py:423]  Training started


Running loss: 5.043767

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


Running loss: 4.698415

...

I0509 22:24:28.330224 140689433306880 configuration_utils.py:142] Configuration saved in CovidBertModel/checkpoint-8000/config.json
I0509 22:24:28.753625 140689433306880 modeling_utils.py:343] Model weights saved in CovidBertModel/checkpoint-8000/pytorch_model.bin


Running loss: 1.654968

I0509 22:28:57.287918 140689433306880 configuration_utils.py:142] Configuration saved in CovidBertModel/checkpoint-8918-epoch-2/config.json


I0509 22:28:57.713877 140689433306880 modeling_utils.py:343] Model weights saved in CovidBertModel/checkpoint-8918-epoch-2/pytorch_model.bin
I0509 22:28:58.506895 140689433306880 configuration_utils.py:142] Configuration saved in CovidBertModel/config.json


I0509 22:28:58.838877 140689433306880 modeling_utils.py:343] Model weights saved in CovidBertModel/pytorch_model.bin
I0509 22:28:58.859014 140689433306880 language_modeling_model.py:352]  Training of bert model complete. Saved to CovidBertModel.


#### Model Retraining

In [20]:
train_args = {
    "output_dir":'CovidBertModel-2',
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    'fp16':False,
    'num_train_epochs':1
}

model = LanguageModelingModel('bert', 'CovidBertModel', args=train_args)
model.tokenizer = model.tokenizer.from_pretrained('CovidBertModel')

model.model.resize_token_embeddings(len(model.tokenizer))

I0511 02:07:50.315353 140372117300992 tokenization_utils.py:420] Model name 'CovidBertModel' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming 'CovidBertModel' is a path, a model identifier, or url to a directory containing tokenizer files.
I0511 02:07:50.317313 140372117300992 tokenization_utils.py:449] Didn't find file CovidBertModel/added_tokens.json. We won't load it.
I0511 02:07:50.318576 140372117300992 tokenization_utils.py:502] loading file CovidBertM

Embedding(30523, 768)

In [ ]:
model.train_model('CovidTrainDf.csv')

I0511 02:08:21.086831 140372117300992 language_modeling_utils.py:177]  Creating features from dataset file at cache_dir/


I0511 02:08:29.440690 140372117300992 language_modeling_utils.py:225]  Saving features into cached file cache_dir/bert_cached_lm_126_CovidTrainDf.csv
I0511 02:08:29.579044 140372117300992 language_modeling_model.py:423]  Training started


I0511 02:08:29.598817 140372117300992 language_modeling_model.py:454]    Starting fine-tuning.


Running loss: 2.434196

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


...

Running loss: 2.102848

I0511 02:18:17.424548 140372117300992 configuration_utils.py:142] Configuration saved in CovidBertModel-2/checkpoint-2000/config.json
I0511 02:18:18.743108 140372117300992 modeling_utils.py:343] Model weights saved in CovidBertModel-2/checkpoint-2000/pytorch_model.bin
/home/roshansk/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running loss: 1.850262

I0511 02:28:10.293651 140372117300992 configuration_utils.py:142] Configuration saved in CovidBertModel-2/checkpoint-4000/config.json
I0511 02:28:11.272383 140372117300992 modeling_utils.py:343] Model weights saved in CovidBertModel-2/checkpoint-4000/pytorch_model.bin


Running loss: 1.788324

### Embedding Testing

In [9]:
model = BertForSequenceClassification.from_pretrained('CovidBertModel-2', output_hidden_states= True)

I0511 01:50:38.372138 140372117300992 configuration_utils.py:281] loading configuration file CovidBertModel-2/config.json
I0511 01:50:38.374210 140372117300992 configuration_utils.py:319] Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "model_type": "bert",
  "no_repeat_ngram_size": 0,
  "num_attenti

In [11]:
model.resize_token_embeddings(30533)

Embedding(30533, 768)

In [12]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30533, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru